In [4]:
import tkinter as tk
from tkinter import messagebox
import random
import time

def draw_bins(canvas, bins, bin_capacity):
    canvas.delete("all")  # Reset the canvas for new output
    x_offset, y_offset = 10, 10
    bin_width, bin_height = 300, 80
    spacing, item_padding = 35, 2
    max_bins_per_row = 3  # Set maximum number of bins per row

    colors = ['red', 'green', 'blue', 'magenta', 'orange', 'grey', 'purple', 'brown']

    for i, bin in enumerate(bins):
        row, col = i // max_bins_per_row, i % max_bins_per_row
        bin_x, bin_y = x_offset + col * (bin_width + spacing), y_offset + row * (bin_height + spacing)
        canvas.create_rectangle(bin_x, bin_y, bin_x + bin_width, bin_y + bin_height, outline="black", fill="white")
        canvas.create_text(bin_x + bin_width + 5, bin_y + bin_height / 2, text=f"Bin {i+1}", anchor='w')

        current_x = bin_x + item_padding
        for item in bin:
            item_width = (bin_width - 2 * item_padding) * (item / bin_capacity)
            item_color = random.choice(colors)
            canvas.create_rectangle(current_x, bin_y + item_padding, current_x + item_width, bin_y + bin_height - item_padding,
                                    fill=item_color, outline="black")
            canvas.create_text(current_x + item_width / 2, bin_y + bin_height / 2, text=str(item), fill="white")
            current_x += item_width + item_padding

def backtracking_solver(items, bins, item_index, capacity, best_solution):
    if item_index == len(items):  
        current_bins_used = len([b for b in bins if sum(b) > 0])
        if current_bins_used < best_solution[0]:  # Find a better solution
            best_solution[0], best_solution[1] = current_bins_used, [list(b) for b in bins if b]
        return

    for bin_index in range(len(bins)):
        if sum(bins[bin_index]) + items[item_index] <= capacity:  # Item fits in the bin
            bins[bin_index].append(items[item_index])
            backtracking_solver(items, bins, item_index + 1, capacity, best_solution)
            bins[bin_index].pop()  # Backtrack

        if not bins[bin_index]:  # Prevent empty bin duplicates
            break

def solve_bin_packing(items, bin_capacity):
    bins = [[] for _ in range(len(items))]  # Initialize with max possible bins
    best_solution = [float('inf'), []]  
    backtracking_solver(items, bins, 0, bin_capacity, best_solution)
    return best_solution[1]

def genetic_algorithm(items, bin_capacity):
    population_size, generations, mutation_rate = 100, 400, 0.1

    def fitness(chromosome):
        bins = {}
        for gene_index, gene in enumerate(chromosome):
            bins[gene] = bins.get(gene, 0) + items[gene_index]
            if bins[gene] > bin_capacity:  # Configuration exceeds bin capacity, check input
                return float('inf')
        return len(set(bins.keys()))  # Fitness is the number of unique bins used

    def crossover(parent1, parent2):
        point = random.randint(1, len(parent1) - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

    def mutate(chromosome):
        for i in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[i] = random.randint(0, max(set(chromosome), default=-1) + 1)  # Limit to existing or next new bin

    population = [[random.randint(0, max(items)//bin_capacity) for _ in items] for _ in range(population_size)]
    for _ in range(generations):
        population.sort(key=fitness)  # Sort by fitness
        next_generation = population[:2]  # carry forward the best solutions

        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population[:20], 2)  # Top 20% of population breeding
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            next_generation.extend([child1, child2])

        population = next_generation

    best_solution = min(population, key=fitness)
    bins = [[] for _ in range(max(best_solution) + 1)]
    for item_index, bin_index in enumerate(best_solution):
        bins[bin_index].append(items[item_index])
    return bins

def run_solver():
    items_str, bin_capacity = items_entry.get(), int(bin_capacity_entry.get())
    algorithm = algo_var.get()

    try:
        items = list(map(int, items_str.split(',')))
    except ValueError:
        messagebox.showerror("Input error", "Ensure all items are integers separated by commas.")
        return

    start_time = time.time()

    if algorithm == 'Backtracking':
        bins = solve_bin_packing(items, bin_capacity)
    elif algorithm == 'Genetic':
        bins = genetic_algorithm(items, bin_capacity)
    else:
        messagebox.showerror("Selection error", "Please select an algorithm.")
        return

    elapsed_time = time.time() - start_time
    draw_bins(canvas, bins, bin_capacity)
    result_label.config(text=f"Minimum bins required using {algorithm}: {len(bins)}")
    time_label.config(text=f"Computation time: {elapsed_time:.2f} seconds")

root = tk.Tk()
root.title("Bin Packing Problem Solver")

tk.Label(root, text="Enter items (comma-separated):").pack()
items_entry = tk.Entry(root)
items_entry.pack()

tk.Label(root, text="Enter bin capacity:").pack()
bin_capacity_entry = tk.Entry(root)
bin_capacity_entry.pack()

algo_var = tk.StringVar()
tk.Radiobutton(root, text="Backtracking", variable=algo_var, value="Backtracking").pack()
tk.Radiobutton(root, text="Genetic Algorithm", variable=algo_var, value="Genetic").pack()

tk.Button(root, text="Solve", command=run_solver).pack()

result_label = tk.Label(root, text="")
result_label.pack()

time_label = tk.Label(root, text="")
time_label.pack()

canvas = tk.Canvas(root, width=1200, height=600)
canvas.pack()

root.mainloop()